# 1. Повышение производительности используемого кода (точнее - циклов, которые тормозят Python)

In [1]:
import numpy as np
import pandas as pd

### Функция, которая вычисляет расстояние между двумя точками на планете Земля по ширине и долготе точек:

In [2]:
def haversine(lat1, lon1, lat2, lon2):
    MILES = 3959
    lat1, lon1, lat2, lon2 = map(np.deg2rad, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    total_miles = MILES * c
    return total_miles

In [3]:
df = pd.read_csv('Data/new_york_hotels.csv', encoding='cp1252')
df.head()

,ean_hotel_id,name,address1,city,state_province,postal_code,latitude,longitude,star_rating,high_rate,low_rate
0,269955,Hilton Garden Inn Albany/SUNY Area,1389 Washington Ave,Albany,NY,12206,42.68751,-73.81643,3.0,154.0272,124.0216
1,113431,Courtyard by Marriott Albany Thruway,1455 Washington Avenue,Albany,NY,12206,42.68971,-73.82021,3.0,179.0100,134.0000
2,108151,Radisson Hotel Albany,205 Wolf Rd,Albany,NY,12205,42.72410,-73.79822,3.0,134.1700,84.1600
3,254756,Hilton Garden Inn Albany Medical Center,62 New Scotland Ave,Albany,NY,12208,42.65157,-73.77638,3.0,308.2807,228.4597
4,198232,CrestHill Suites SUNY University Albany,1415 Washington Avenue,Albany,NY,12206,42.68873,-73.81854,3.0,169.3900,89.3900


### Функция, которая вычисляет список расстояний между точкой на карте США и отелями Нью-Йорка:

In [5]:
def haversine_looping(df):
    distance_list = []
    for i in range(0, len(df)):
        d = haversine(40.671, -73.985, df.iloc[i]['latitude'], df.iloc[i]['longitude'])
        distance_list.append(d)
    return distance_list

In [6]:
%%timeit
# запускаем итеративную функцию haversine
df['distance'] = haversine_looping(df)

3.42 s ± 88.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
# запускаем итерирование с помощью
# метода .iterrrows()
haversine_series = []
for index, row in df.iterrows():
    haversine_series.append(haversine(40.671, -73.985, row['latitude'], row['longitude']))
df['distance'] = haversine_series

1.41 s ± 108 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
# применяем функцию haversine с помощью
# метода .apply()
df['distance'] = df.apply(lambda row: haversine(40.671, -73.985, row['latitude'],
                            row['longitude']), axis=1)

312 ms ± 18.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%%timeit
# векторизированная реализация функции haversine,
# используем объекты Series целиком
df['distance'] = haversine(40.671, -73.985,df['latitude'], df['longitude'])

19.6 ms ± 2.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
# векторизированная реализация функции haversine,
# используем вместо объектов Series массивы NumPy
df['distance'] = haversine(40.671, -73.985, df['latitude'].values, df['longitude'].values)

2.48 ms ± 442 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Таким образом, мы постепенно достигли увеличения скорости кода в 3,42*1000 / 2,48 = 1380 раз. То есть надо при составлении кода с итерациями делать так, чтобы выполняемые функции в итерациях были векторизованы на основе массивов NumPy (например, `'df.values'`).

# 2. Применение Cython для повышения производительности кода

In [11]:
# импортируем numpy и pandas
import numpy as np
import pandas as pd
np.random.seed(12345)
df = pd.DataFrame({'a': np.random.randn(1000),
'b': np.random.randn(1000),
'N': np.random.randint(100, 1000, (1000)),
'x': 'x'})

In [12]:
df

,a,b,N,x
0,-0.204708,-0.983505,826,x
1,0.478943,0.930944,220,x
2,-0.519439,-0.811676,401,x
3,-0.555730,-1.830156,363,x
4,1.965781,-0.138730,235,x
...,...,...,...,...
995,0.107657,-1.341493,700,x
996,-0.139298,-0.293333,211,x
997,-1.159926,-0.242459,315,x
998,0.618965,-3.056990,279,x


In [13]:
def f(x):
    return x * (x - 1)

def integrate_f(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f(a + i * dx)
    return s * dx

In [14]:
%timeit df.apply(lambda x: integrate_f(x['a'], x['b'], x['N']), axis=1)

529 ms ± 6.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Смотрим, на что тратится основное время при выполнении кода:

In [15]:
%prun -l 4 df.apply(lambda x: integrate_f(x['a'], x['b'], x['N']), axis=1)

### Загружаем Cython (надо его предварительно установить через pip). Код остается таким же, просто добавляется декоратор `%%cython`:

In [16]:
%load_ext Cython

In [17]:
%%cython
def f_plain(x):
    return x * (x - 1)

def integrate_f_plain(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f_plain(a + i * dx)
    return s * dx

In [18]:
%timeit df.apply(lambda x: integrate_f_plain(x['a'], x['b'], x['N']), axis=1)

324 ms ± 9.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# 3. Применение Numba для повышения производительности кода (код конечно видоизменить):

In [19]:
import numba
@numba.jit
def f_plain(x):
    return x * (x - 1)

In [20]:
@numba.jit
def integrate_f_numba(a, b, N):
    s = 0
    dx = (b - a) / N
    for i in range(N):
        s += f_plain(a + i * dx)
    return s * dx

@numba.jit
def apply_integrate_f_numba(col_a, col_b, col_N):
    n = len(col_N)
    result = np.empty(n, dtype='float64')
    assert len(col_a) == len(col_b) == n
    for i in range(n):
        result[i] = integrate_f_numba(col_a[i], col_b[i], col_N[i])
    return result

def compute_numba(df):
    result = apply_integrate_f_numba(df['a'].values, df['b'].values, df['N'].values)
    return pd.Series(result, index=df.index, name='result')

In [21]:
%timeit compute_numba(df)

4.99 ms ± 139 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Мы достигли 529/4,99 = 100-кратного увеличения скорости. Даже по сравнению с векторизованной формой.

# 4. ИСПОЛЬЗУЕМ PANDAS ДЛЯ БОЛЬШИХ ДАННЫХ. Оптимизация памяти и типов используемых данных в `DataFrame`.

In [22]:
# отключаем предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')
# импортируем библиотеку pandas
import pandas as pd
# cчитываем данные и выводим первые 5 наблюдений
gl = pd.read_csv('Data/game_logs.csv')
gl.head()

,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,18710504,0,Thu,CL1,na,1,FW1,na,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,18710505,0,Fri,BS1,na,1,WS3,na,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,18710506,0,Sat,CL1,na,2,RC1,na,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,18710508,0,Mon,CL1,na,3,CH1,na,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,18710509,0,Tue,BS1,na,2,TRO,na,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y


In [23]:
# выводим точную информацию об использовании памяти
gl.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: float64(77), int64(6), object(78)
memory usage: 859.4 MB


In [24]:
# посмотрим, сколько памяти в среднем используют
# столбцы определенного типа
for dtype in ['float','int','object']:
    selected_dtype = gl.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Использование памяти в среднем для {} столбцов: {:03.2f} MB".format(dtype,mean_usage_mb))

Использование памяти в среднем для float столбцов: 1.29 MB
Использование памяти в среднем для int столбцов: 1.12 MB
Использование памяти в среднем для object столбцов: 9.50 MB


In [25]:
# мы будем довольно часто подсчитывать использование памяти,
# поэтому напишем функцию, которая сэкономит нам немного времени
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # предположим, что если это не датафрейм, то серия
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # преобразуем байты в мегабайты
    return "{:03.2f} MB".format(usage_mb)
# выполняем понижающее преобразование
# для столбцов типа int
gl_int = gl.select_dtypes(include=['int'])
converted_int = gl_int.apply(pd.to_numeric,downcast='unsigned')
print(mem_usage(gl_int))
print(mem_usage(converted_int))

7.87 MB
1.48 MB


In [26]:
compare_ints = pd.concat([gl_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

,before,after
uint8,NaN,5.0
uint32,NaN,1.0
int64,6.0,NaN


In [27]:
# выполняем понижающее преобразование
# для столбцов типа float
gl_float = gl.select_dtypes(include=['float'])
converted_float = gl_float.apply(pd.to_numeric,downcast='float')
print(mem_usage(gl_float))
print(mem_usage(converted_float))
compare_floats = pd.concat([gl_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

100.99 MB
50.49 MB


,before,after
float32,NaN,77.0
float64,77.0,NaN


In [28]:
# создаем копию исходного датафрейма
optimized_gl = gl.copy()
# заменяем исходные числовые столбцы
# оптимизированными
optimized_gl[converted_int.columns] = converted_int
optimized_gl[converted_float.columns] = converted_float
# смотрим использование памяти
print(mem_usage(gl))
print(mem_usage(optimized_gl))

859.43 MB
802.54 MB


In [29]:
# смотрим количество уникальных значений
# по каждому столбцу типа object
gl_obj = gl.select_dtypes(include=['object']).copy()
gl_obj.describe()

,day_of_week,v_name,v_league,h_name,h_league,day_night,completion,forefeit,protest,park_id,...,h_player_6_id,h_player_6_name,h_player_7_id,h_player_7_name,h_player_8_id,h_player_8_name,h_player_9_id,h_player_9_name,additional_info,acquisition_info
count,171907,171907,171907,171907,171907,140150,116,145,180,171907,...,140838,140838,140838,140838,140838,140838,140838,140838,1456,140841
unique,7,148,7,148,7,2,116,3,5,245,...,4774,4720,5253,5197,4760,4710,5193,5142,332,1
top,Sat,CHN,NL,CHN,NL,D,"20150718,,2,3,30",H,V,STL07,...,grimc101,Charlie Grimm,grimc101,Charlie Grimm,lopea102,Al Lopez,spahw101,Warren Spahn,HTBF,Y
freq,28891,8870,88866,9024,88867,82724,1,69,90,7022,...,427,427,491,491,676,676,339,339,1112,140841


In [30]:
# запишем столбец day_of_week как dow
dow = gl_obj.day_of_week
print(dow.head())

0    Thu
1    Fri
2    Sat
3    Mon
4    Tue
Name: day_of_week, dtype: object


### Эффектный финт. Переводим при ограниченном кол-ве значений столбца тип данных из `object` в `category`:

In [31]:
# присваиваем столбцу dow тип category
dow_cat = dow.astype('category')
print(dow_cat.head())

0    Thu
1    Fri
2    Sat
3    Mon
4    Tue
Name: day_of_week, dtype: category
Categories (7, object): ['Fri', 'Mon', 'Sat', 'Sun', 'Thu', 'Tue', 'Wed']


In [32]:
print(mem_usage(dow))
print(mem_usage(dow_cat))


9.84 MB
0.16 MB


In [33]:
converted_obj = pd.DataFrame()
# пишем цикл, которой перебирает каждый столбец object,
# проверяет его на соответствие заданному порогу
# (количество уникальных значений должно быть меньше 50%
# от общего количества значений), и если столбец
# удовлетворяет порогу, преобразовывает его в тип category
for col in gl_obj.columns:
    num_unique_values = len(gl_obj[col].unique())
    num_total_values = len(gl_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = gl_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = gl_obj[col]

In [34]:
# снова применяем функцию mem_usage, смотрим,
# сколько памяти занимают все столбцы типа object
# до и после преобразования в тип category
print(mem_usage(gl_obj))
print(mem_usage(converted_obj))
compare_obj = pd.concat([gl_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

750.57 MB
49.89 MB


,before,after
object,78.0,NaN
category,NaN,2.0
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0
...,...,...
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0


In [35]:
# смотрим, сколько памяти использует датафрейм
# после оптимизации типов
optimized_gl[converted_obj.columns] = converted_obj
mem_usage(optimized_gl)

'101.86 MB'

### Эффектный финт номер 2. Задаем типы во время считывания данных. То есть мы предварительно изучаем, какие столбцы могут быть оптимизированы при загрузке путем указания категориального или числового типа. И явно в параметрах указываем, что надо считывать такой-то столбец как определенный тип данных:

In [36]:
# создаем словарь, в котором ключами будут имена
# столбцов, а значениями - типы столбцов
dtypes = optimized_gl.drop('date',axis=1).dtypes
dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]
column_types = dict(zip(dtypes_col, dtypes_type))
# вместо того, чтобы печатать 161 столбец,
# выберем 10 пар "ключ-значение" из словаря
# и красиво распечатаем их, используя prettyprint
preview = first2pairs = {key:value for key,value in list(column_types.items())[:10]}
import pprint
pp = pp = pprint.PrettyPrinter(indent=4)
pp.pprint(preview)

{   'day_of_week': 'category',
    'h_game_number': 'uint8',
    'h_league': 'category',
    'h_name': 'category',
    'h_score': 'uint8',
    'number_of_game': 'uint8',
    'v_game_number': 'uint8',
    'v_league': 'category',
    'v_name': 'category',
    'v_score': 'uint8'}


In [38]:
# считываем данные с нужными нам типами
read_and_optimized = pd.read_csv('Data/game_logs.csv', dtype=column_types,
                                 parse_dates=['date'], infer_datetime_format=True)
# смотрим использование памяти
print(mem_usage(read_and_optimized))
read_and_optimized.head()

102.51 MB


,date,number_of_game,day_of_week,v_name,v_league,v_game_number,h_name,h_league,h_game_number,v_score,...,h_player_7_name,h_player_7_def_pos,h_player_8_id,h_player_8_name,h_player_8_def_pos,h_player_9_id,h_player_9_name,h_player_9_def_pos,additional_info,acquisition_info
0,1871-05-04,0,Thu,CL1,na,1,FW1,na,1,0,...,Ed Mincher,7.0,mcdej101,James McDermott,8.0,kellb105,Bill Kelly,9.0,NaN,Y
1,1871-05-05,0,Fri,BS1,na,1,WS3,na,1,20,...,Asa Brainard,1.0,burrh101,Henry Burroughs,9.0,berth101,Henry Berthrong,8.0,HTBF,Y
2,1871-05-06,0,Sat,CL1,na,2,RC1,na,1,12,...,Pony Sager,6.0,birdg101,George Bird,7.0,stirg101,Gat Stires,9.0,NaN,Y
3,1871-05-08,0,Mon,CL1,na,3,CH1,na,1,12,...,Ed Duffy,6.0,pinke101,Ed Pinkham,5.0,zettg101,George Zettlein,1.0,NaN,Y
4,1871-05-09,0,Tue,BS1,na,2,TRO,na,1,9,...,Steve Bellan,5.0,pikel101,Lip Pike,3.0,cravb101,Bill Craver,6.0,HTBF,Y


### Таким образом, мы в 8 раз снизили размер загружаемого файла.